---
# **문서 유사도(Document Similarity)**
문서의 유사도를 구하는 일은 자연어 처리의 주요 주제 중 하나입니다. 사람들이 인식하는 문서의 유사도란 주로 문서들 간에 동일한 단어 또는 비슷한 단어가 얼마나 공통적으로 많이 사용되었는지에 달려있습니다. 기계도 마찬가지입니다. ***기계가 계산하는 문서의 유사도의 성능은 각 문서의 단어들을 어떤 방법으로 수치화하여 표현했는지(DTM, Word2Vec 등), 문서 간의 단어들의 차이를 어떤 방법(유클리드 거리, 코사인 유사도 등)으로 계산했는지에 달려있습니다.***

# **1) 코사인 유사도(Cosine Similarity)**
BoW나 BoW에 기반한 단어 표현 방법인 DTM, TF-IDF, 또는 뒤에서 배우게 될 워드투벡터(Word2Vec) 등과 같이 단어를 수치화할 수 있는 방법을 이해했다면, 이러한 표현 방법에 대해서 코사인 유사도를 이용하여 문서의 유사도를 구하는 게 가능합니다.

## **1. 코사인 유사도(Cosine Similarity)**

코사인 유사도는 두 벡터 간의 코사인 각도를 이용하여 구할 수 있는 두 벡터의 유사도를 의미합니다. 두 벡터의 방향이 완전히 동일한 경우에는 1의 값을 가지며, 90°의 각을 이루면 0, 180°로 반대의 방향을 가지면 -1의 값을 갖게 됩니다. 즉, 결국 코사인 유사도는 -1 이상 1 이하의 값을 가지며 값이 1에 가까울수록 유사도가 높다고 판단할 수 있습니다. 이를 직관적으로 이해하면 두 벡터가 가리키는 방향이 얼마나 유사한가를 의미합니다.

![image](https://user-images.githubusercontent.com/42408554/63221223-d1e03180-c1d0-11e9-91e2-189e7f914141.png)

두 벡터 A, B에 대해서 코사인 유사도는 식으로 표현하면 다음과 같습니다.

![image](https://user-images.githubusercontent.com/42408554/63221226-e45a6b00-c1d0-11e9-82d8-d88827f39e13.png)

문서 단어 행렬(DTM)이나 TF-IDF 행렬을 통해서 문서의 유사도를 구하는 경우에는 문서 단어 행렬이나 TF-IDF 행렬이 각각의 특징 벡터 A, B가 됩니다. 그렇다면 문서 단어 행렬에 대해서 코사인 유사도를 구해보는 간단한 예제를 진행해보겠습니다.

> 문서1 : 저는 사과 좋아요 <br>
> 문서2 : 저는 바나나 좋아요 <br>
> 문서3 : 저는 바나나 좋아요 저는 바나나 좋아요 <br>

위의 세 문서에 대해서 ***문서 단어 행렬(DTM)***을 만들면 이와 같습니다.

|   -   | 바나나 | 사과 | 저는 | 좋아요 |
| :---: | :----: | :--: | :--: | :----: |
| 문서1 |   0    |  1   |  1   |   1    |
| 문서2 |   1    |  0   |  1   |   1    |
| 문서3 |   2    |  0   |  2   |   2    |

파이썬에서는 코사인 유사도를 구하는 방법은 여러가지가 있는데 여기서는 Numpy를 이용해서 계산해보겠습니다.

In [1]:
from numpy import dot
from numpy.linalg import norm
import numpy as np

def cos_sim(A, B):
    return dot(A, B)/(norm(A) * norm(B))

위에서 코사인 유사도를 계산하는 함수를 만들었습니다.

In [3]:
doc1 = np.array([0,1,1,1])
doc2 = np.array([1,0,1,1])
doc3 = np.array([2,0,2,2])

위는 예를 들었던 문서1, 문서2, 문서3에 대해서 각각 BoW를 만든 내용입니다. 이제 각 문서에 대한 코사인 유사도를 계산해보겠습니다.

In [4]:
# 문서1 : 저는 사과 좋아요
# 문서2 : 저는 바나나 좋아요
# 문서3 : 저는 바나나 좋아요 저는 바나나 좋아요

print(cos_sim(doc1, doc2))
print(cos_sim(doc1, doc3))
print(cos_sim(doc2, doc3))

0.6666666666666667
0.6666666666666667
1.0000000000000002


눈여겨볼만한 점은 문서1과 문서2의 코사인 유사도와 문서1과 문서3의 코사인 유사도가 같다는 점과 ***문서2와 문서3의 코사인 유사도가 1***이 나온다는 것입니다. 앞서 1은 두 벡터의 방향이 완전히 동일한 경우에 1이 나오며, 코사인 유사도 관점에서는 유사도의 값이 최대임을 의미한다고 언급한 바 있습니다.

***문서3은 문서2에서 단지 모든 단어의 빈도수가 1씩 증가했을 뿐입니다. 이는 한 문서 내의 모든 단어의 빈도수가 똑같이 증가하는 경우에는 기존의 문서와 코사인 유사도의 값이 1이라는 것***입니다.

## **2. 유사도를 이용한 추천 시스템 구현하기**

캐글에서 사용되었던 영화 데이터셋을 가지고 영화 추천 시스템을 만들어보겠습니다. TF-IDF와 코사인 유사도만으로 영화의 줄거리에 기반해서 영화를 추천하는 추천 시스템을 만들 수 있습니다.

- 다운로드 링크 : https://www.kaggle.com/rounakbanik/the-movies-dataset

원본 파일은 위 링크에서 movies_metadata.csv 파일을 다운로드 받으면 됩니다. 해당 데이터는 총 24개의 열을 가진 45,466개의 샘플로 구성된 영화 정보 데이터입니다.

In [5]:
import pandas as pd
data = pd.read_csv('C:/Python/NLP/NLP_BASIC/dataset/movies_metadata.csv',
                  low_memory = False)

data.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


우선 다운로드 받은 훈련 데이터에서 2개의 샘플만 출력하여, 데이터가 어떤 형식을 갖고있는지 확인합니다. csv 파일을 읽기 위해서 파이썬 라이브러리인 pandas를 갖고왔는데, pandas는 뒤에서도 pd란 이름으로 계속해서 유용하게 쓰이게 됩니다.

훈련 데이터는 총 24개의 열을 갖고있으나, 책의 지면상 일부 생략하였습니다. 여기서 코사인 유사도에 사용할 데이터는 영화 제목에 해당하는 title 열과 줄거리에 해당하는 overview 열입니다. 좋아하는 영화를 입력하면, 해당 영화의 줄거리와 줄거리가 유사한 영화를 찾아서 추천하는 시스템을 만들 것입니다.

In [6]:
data = data.head(20000)

만약 훈련 데이터의 양을 줄이고 학습을 진행하고자 한다면, 이와 같이 데이터를 줄여서 재저장할 수 있습니다. 저는 20,000개의 샘플만 가지고 학습해보겠습니다. ***tf-idf를 할 때 데이터에 Null 값이 들어있으면 에러가 발생***합니다. tf-idf의 대상이 되는 data의 overview 열에 ***Null 값이 있는지 확인***합니다.

In [8]:
data['overview'].isnull().sum()

135

135개의 샘플에서 Null 값이 있다고 합니다. pandas를 이용하면 ***Null 값을 처리하는 도구인 fillna()를 사용***할 수 있습니다. 괄호 안에 Null 대신 넣고자하는 값을 넣으면 되는데, 이 경우에는 빈 값(empty value)으로 대체하여 Null 값을 제거합니다.

In [9]:
data['overview'] = data['overview'].fillna('') # overview에서 Null 값을 가진 경우에는 값 제거

Null 값을 제거하였습니다. 이제 .isnull().sum()를 수행하면 0의 값이 나옵니다. 이제 tf-idf를 수행합니다.

In [10]:
data['overview'].isnull().sum()

0

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words = 'english')
tfidf_matrix = tfidf.fit_transform(data['overview']) # overview에 대해서 tf-idf 수행

print(tfidf_matrix.shape)

(20000, 47487)


overview 열에 대해서 tf-idf를 수행했습니다. 20,000개의 영화를 표현하기위해 총 47,487개의 단어가 사용되었음을 보여주고 있습니다. 이제 코사인 유사도를 사용하면 바로 문서의 유사도를 구할 수 있습니다.

In [12]:
from sklearn.metrics.pairwise import linear_kernel

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

코사인 유사도를 구합니다.

In [13]:
indices = pd.Series(data.index, index = data['title']).drop_duplicates()
print(indices.head())

title
Toy Story                      0
Jumanji                        1
Grumpier Old Men               2
Waiting to Exhale              3
Father of the Bride Part II    4
dtype: int64


영화의 타이틀과 인덱스를 가진 테이블을 만듭니다. 이 중 5개만 출력해보도록 하겠습니다. 이 테이블의 용도는 영화의 타이틀을 입력하면 인덱스를 리턴하기 위함입니다.

In [14]:
idx = indices['Father of the Bride Part II']
print(idx)

4


이제 선택한 영화에 대해서 코사인 유사도를 이용하여, 가장 overview가 유사한 10개의 영화를 찾아내는 함수를 만듭니다.

In [15]:
def get_recommendations(title, cosine_sim = cosine_sim):
    # 선택한 영화의 타이틀로부터 해당되는 인덱스를 받아옵니다. 이제 선택한 영화를 가지고 연산할 수 있습니다.
    idx = indices[title]
    
    # 모든 영화에 대해서 해당 영화와의 유사도를 구합니다.
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    # 유사도에 따라 영화들을 정렬합니다.
    sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)
    
    # 가장 유사한 10개의 영화를 받아옵니다.
    sim_scores = sim_scores[1:11]
    
    # 가장 유사한 10개의 영화의 인덱스를 받아옵니다.
    movie_indices = [i[0] for i in sim_scores]
    
    # 가장 유사한 10개의 영화의 제목을 리턴합니다.
    return data['title'].iloc[movie_indices]

영화 다크 나이트 라이즈와 overview가 유사한 영화들을 찾아보겠습니다.

In [25]:
get_recommendations('The Dark Knight Rises')

12481                            The Dark Knight
150                               Batman Forever
1328                              Batman Returns
15511                 Batman: Under the Red Hood
585                                       Batman
9230          Batman Beyond: Return of the Joker
18035                           Batman: Year One
19792    Batman: The Dark Knight Returns, Part 1
3095                Batman: Mask of the Phantasm
10122                              Batman Begins
Name: title, dtype: object

가장 유사한 영화가 출력되는데, 영화 다크 나이트가 첫번째고, 그 외에도 전부 배트맨 영화를 찾아낸 것을 확인할 수 있습니다.

---
# 2) 여러가지 유사도 기법

문서의 유사도를 구하기 위한 방법으로는 코사인 유사도 외에도 여러가지 방법들이 있습니다. 여기서는 문서의 유사도를 구할 수 있는 다른 방법들을 학습합니다.

## **1. 유클리드 거리(Euclidean distance)**

유클리드 거리(euclidean distance)는 문서의 유사도를 구할 때 자카드 유사도나 코사인 유사도만큼, 유용한 방법은 아닙니다. 하지만 여러 가지 방법을 이해하고, 시도해보는 것 자체만으로 다른 개념들을 이해할 때 도움이 되므로 의미가 있습니다.

다차원 공간에서 두개의 점 p와 q가 각각 p=(p1,p2,p3,...,pn)과 q=(q1,q2,q3,...,qn)의 좌표를 가질 때 두 점 사이의 거리를 계산하는 유클리드 거리 공식은 다음과 같습니다.

![image](https://user-images.githubusercontent.com/42408554/63231835-5330d600-c25c-11e9-8940-affbd9de9ea7.png)

다차원 공간이라고 가정하면, 처음 보는 입장에서는 식이 너무 복잡해보입니다. 좀 더 쉽게 이해하기위해서 2차원 공간이라고 가정하고 두 점 사이의 거리를 좌표 평면 상에서 시각화해보겠습니다.

![image](https://user-images.githubusercontent.com/42408554/63231882-f97cdb80-c25c-11e9-8c11-e3e5437bb3e4.png)

2차원 좌표 평면 상에서 두 점 pp와 qq사이의 직선 거리를 구하는 문제입니다. 위의 경우에는 직각 삼각형으로 표현이 가능하므로, 중학교 수학 과정인 피타고라스의 정리를 통해 pp와 qq 사이의 거리를 계산할 수 있습니다. 즉, 2차원 좌표 평면에서 두 점 사이의 유클리드 거리 공식은 피타고라스의 정리를 통해 두 점 사이의 거리를 구하는 것과 동일합니다.

다시 원점으로 돌아가서 여러 문서에 대해서 유사도를 구하고자 유클리드 거리 공식을 사용한다는 것은, 앞서 본 2차원을 단어의 총 개수만큼의 차원으로 확장하는 것과 같습니다. 예를 들어 아래와 같은 DTM이 있다고 합시다.

|   -   | 바나나 | 사과 | 저는 | 좋아요 |
| :---: | :----: | :--- | :--: | :----: |
| 문서1 |   2    | 3    |  0   |   1    |
| 문서2 |   1    | 2    |  3   |   1    |
| 문서3 |   2    | 1    |  2   |   2    |

단어의 개수가 4개이므로, 이는 4차원 공간에 문서1, 문서2, 문서3을 배치하는 것과 같습니다. 이때 다음과 같은 문서Q에 대해서 문서1, 문서2, 문서3 중 가장 유사한 문서를 찾아내고자 합니다.

|   -   | 바나나 | 사과 | 저는 | 좋아요 |
| :---: | :----: | :--: | :--: | :----: |
| 문서Q |   1    |  1   |  0   |   1    |

이때 유클리드 거리를 통해 유사도를 구하려고 한다면, 문서Q 또한 다른 문서들처럼 4차원 공간에 배치시켰다는 관점에서 4차원 공간에서의 각각의 문서들과의 유클리드 거리를 구하면 됩니다. 이를 파이썬 코드로 구현해보겠습니다.

In [2]:
import numpy as np

def dist(x ,y):
    return np.sqrt(np.sum((x-y) ** 2)) # sqrt 제곱근(루트)을 계산함. 

doc1 = np.array((2, 3, 0, 1))
doc2 = np.array((1, 2, 3, 1))
doc3 = np.array((2, 1, 2, 2))
docQ = np.array((1, 1, 0, 1))

print(dist(doc1, docQ))
print(dist(doc2, docQ))
print(dist(doc3, docQ))

2.23606797749979
3.1622776601683795
2.449489742783178


유클리드 거리의 값이 가장 작다는 것은, 문서 간의 거리가 가장 가깝다는 것을 의미합니다. 즉, 문서1이 문서Q와 가장 유사하다고 볼 수 있습니다.

## **2. 자카드 유사도(Jaccard similarity)**
A와 B 두개의 집합이 있다고 합시다. 이때 교집합은 두 개의 집합에서 공통으로 가지고 있는 원소들의 집합을 말합니다. 즉, 합집합에서 교집합의 비율을 구한다면 두 집합 A와 B의 유사도를 구할 수 있다는 것이 자카드 유사도(jaccard similarity)의 아이디어입니다. 자카드 유사도는 0과 1사이의 값을 가지며, 만약 두 집합이 동일하다면 1의 값을 가지고, 두 집합의 공통 원소가 없다면 0의 값을 갖습니다. 자카드 유사도를 구하는 함수를 JJ라고 하였을 때, 자카드 유사도 함수 JJ는 아래와 같습니다.

![image](https://user-images.githubusercontent.com/42408554/63232003-647ae200-c25e-11e9-8f7f-c6b39dc50d6a.png)

두 개의 비교할 문서를 각각 doc1doc1, doc2doc2라고 했을 때 doc1doc1과 doc2doc2의 문서의 유사도를 구하기 위한 자카드 유사도는 이와 같습니다.

![image](https://user-images.githubusercontent.com/42408554/63232017-7fe5ed00-c25e-11e9-8cc3-97dc86461ec7.png)

즉, 두 문서 doc1, doc1 사이의 자카드 유사도 J(doc1,doc2)는 두 집합의 교집합 크기를 두 집합의 합집합 크기로 나눈 값으로 정의됩니다. 간단한 예를 통해서 이해해보겠습니다.

In [7]:
# 다음과 같은 두 개의 문서가 있습니다.
# 두 문서 모두에서 등장한 단어는 apple과 banana 2개.
doc1 = "apple banana everyone like likey watch card holder"
doc2 = "apple banana coupon passport love you"

# 토큰화를 수행합니다.
tokenized_doc1 = doc1.split()
tokenized_doc2 = doc2.split()

# 토큰화 결과 출력
print(tokenized_doc1)
print(tokenized_doc2)

['apple', 'banana', 'everyone', 'like', 'likey', 'watch', 'card', 'holder']
['apple', 'banana', 'coupon', 'passport', 'love', 'you']


이 때 문서1과 문서2의 합집합을 구해보겠습니다.

In [12]:
union = set(tokenized_doc1).union(set(tokenized_doc2))
print(union)

{'like', 'everyone', 'banana', 'coupon', 'likey', 'holder', 'watch', 'you', 'apple', 'card', 'passport', 'love'}


문서1과 문서2의 합집합의 단어의 총 개수는 12개인 것을 확인할 수 있습니다. 그렇다면, 문서1과 문서2의 교집합을 구해보겠습니다. 즉, 문서1과 문서2에서 둘 다 등장한 단어를 구하게 됩니다.

In [14]:
intersection = set(tokenized_doc1).intersection(set(tokenized_doc2))
print(intersection)

{'apple', 'banana'}


문서1과 문서2에서 둘 다 등장한 단어는 banana와 apple 총 2개입니다. 이제 교집합의 수를 합집합의 수로 나누면 자카드 유사도가 계산됩니다.

In [15]:
print(len(intersection)/len(union))

0.16666666666666666


위의 값은 자카드 유사도이자, 두 문서의 총 단어 집합에서 두 문서에서 공통적으로 등장한 단어의 비율이기도 합니다.